In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

$\theta\sim \mathcal{N}(0, \sigma^2)$


${\rm mmse}^*(\Sigma^{-2})=\frac{\sigma^2}{1+\sigma^2\Sigma^{-2}}$

${\rm mmse}(\Sigma^{-2}, t)={\rm mmse}^*(\Sigma^{-2}+t)=\frac{\sigma^2}{1+\sigma^2(\Sigma^{-2}+t)}$

$\eta(u; \Sigma^2, z_t, t) = \frac{\sigma^2(z_t\Sigma^2+u)}{\Sigma^2+\sigma^2+t\sigma^2\Sigma^2}$

$\eta'(u; \Sigma^2, z_t, t) = \frac{\sigma^2}{\Sigma^2+\sigma^2+t\sigma^2\Sigma^2}$

In [ ]:
def get_MMSE(inverse_Sigma_squared, t, prior_sigma_squared):
    res = prior_sigma_squared / (1 + prior_sigma_squared * (inverse_Sigma_squared + t))
    return res



def get_tau_t_squared_series(t, Delta, nu, alpha, K, prior_sigma_squared):
    tau_squared_series = np.zeros(K)
    tau_squared_series[0] = (Delta + nu) / alpha
    for k in range(1, K):
        tau_squared_series[k] = (Delta + get_MMSE((tau_squared_series[k-1]) ** (-1), t, prior_sigma_squared))/ alpha
    return tau_squared_series



def get_eta(u, Sigma_squared, z_t, t, prior_sigma_squared):
    num = prior_sigma_squared * (z_t * Sigma_squared + u)
    deno = Sigma_squared + prior_sigma_squared + t * prior_sigma_squared * Sigma_squared
    return num / deno
        


def get_eta_prime(u, Sigma_squared, z_t, t, prior_sigma_squared):
    num = prior_sigma_squared
    deno = Sigma_squared + prior_sigma_squared + t * prior_sigma_squared * Sigma_squared
    return [num/deno] * len(u)


def run_AMP(y, z_t, t, phi_matrix, Delta, nu, alpha, K, prior_sigma_squared):
    N = phi_matrix.shape[1]
    m = np.zeros(N)
    r = y
    tau_squared_series = get_tau_t_squared_series(t, Delta, nu, alpha, K, prior_sigma_squared)
    for k in range(K):
        m = get_eta(np.dot(phi_matrix.T, r) + m, tau_squared_series[k], z_t, t, prior_sigma_squared)
        b = sum(get_eta_prime(np.dot(phi_matrix.T, r) + m, tau_squared_series[k], z_t, t, prior_sigma_squared)) / (N * alpha)
        r = y - np.dot(phi_matrix, m) + r * b
    return m


def run_alg(y, phi_matrix, K, T, delta, Delta, nu, alpha, prior_sigma_squared):
    N = phi_matrix.shape[1]
    L = int(T / delta)
    theta_alg_timeline = np.zeros((L, N))
    z_t_tilde = np.zeros(N)
    theta_alg_timeline[0, :] = z_t_tilde
    for l in range(1, L):
        t = (l+1) * delta
        b_motion = np.random.normal(loc=0, scale=1, size=N)
        m = run_AMP(y, z_t_tilde, t, phi_matrix, Delta, nu, alpha, K, prior_sigma_squared)
        z_t_tilde = z_t_tilde + m * delta + np.sqrt(delta) * b_motion
        theta_alg_timeline[l, :] = z_t_tilde/t
    return theta_alg_timeline



def plot_fir_sec(theta_alg_timeline, display_interval):
    plt.axis([-2, 2, -2, 2])
    plt.scatter(theta_alg_timeline[0, 0], theta_alg_timeline[0, 1], s=50, marker="8", color="orange")
    plt.scatter(theta_alg_timeline[::display_interval, 0], theta_alg_timeline[::display_interval, 1], s=2, color="#529AC9")
    plt.plot(theta_alg_timeline[::display_interval, 0], theta_alg_timeline[::display_interval, 1], linewidth=0.5, color="#529AC9")
    plt.scatter(theta_alg_timeline[-1, 0], theta_alg_timeline[-1, 1], s=200, marker="*", color="#F49600")



def plot_fir_sec_multi(theta_alg_timeline1, theta_alg_timeline2, theta_alg_timeline3, theta_alg_timeline4, display_interval, pth):
    plt.subplot(1, 4, 1)
    plot_fir_sec(theta_alg_timeline1, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(1, 4, 2)
    plot_fir_sec(theta_alg_timeline2, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(1, 4, 3)
    plot_fir_sec(theta_alg_timeline3, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplot(1, 4, 4)
    plot_fir_sec(theta_alg_timeline4, display_interval)
    ax = plt.gca()
    ax.set_aspect(1)
    
    plt.subplots_adjust(wspace=0.5)
    plt.savefig(pth)
    plt.show()


    
def alg_mse(theta, theta_alg):
    mse = np.zeros(len(theta_alg))
    for i in range(len(theta_alg)):
        mse[i] = np.mean((theta - theta_alg[i]) ** 2)
    return mse



def get_y_phi_theta(M, N, Delta, prior_sigma_squared, seed):
    np.random.seed(seed)
    alpha = M/N
    theta = np.random.normal(loc=0.0, scale = np.sqrt(prior_sigma_squared), size=N)
    phi_matrix = np.random.normal(loc=0.0, scale=np.sqrt(1 / M), size=(M, N))
    w = np.random.normal(loc=0.0, scale=1, size=M)
    y = np.dot(phi_matrix, theta) + np.sqrt(Delta/alpha) * w
    return [y, phi_matrix, theta]



In [ ]:
alpha = 2
K = 50
T = 300
delta = 0.1

Delta= 0.01

prior_sigma_squared = 1
nu = prior_sigma_squared

seed = 212

In [ ]:
N1 = 192
M1 = int(N1*alpha)
y1, phi_matrix1, theta1 = get_y_phi_theta(M1, N1, Delta, prior_sigma_squared, seed)


start_time = time.time()
theta_alg_timeline1 = run_alg(y1, phi_matrix1, K, T, delta, Delta, nu, alpha, prior_sigma_squared)
end_time = time.time()
alg_mse1 = alg_mse(theta1, theta_alg_timeline1)
print("execution time: {:.2f}seconds".format(end_time - start_time))

np.save("./data_saved/normal_theta_origin_Delta0_01_seed212_N192", theta1)
np.save("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N192", theta_alg_timeline1)

In [ ]:
N2 = 768
M2 = int(N2*alpha)
y2, phi_matrix2, theta2 = get_y_phi_theta(M2, N2, Delta, prior_sigma_squared, seed)


start_time = time.time()
theta_alg_timeline2 = run_alg(y2, phi_matrix2, K, T, delta, Delta, nu, alpha, prior_sigma_squared)
end_time = time.time()
alg_mse2 = alg_mse(theta2, theta_alg_timeline2)
print("execution time: {:.2f}seconds".format(end_time - start_time))

np.save("./data_saved/normal_theta_origin_Delta0_01_seed212_N768", theta2)
np.save("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N768", theta_alg_timeline2)

In [ ]:
N3 = 1728
M3 = int(N3*alpha)
y3, phi_matrix3, theta3 = get_y_phi_theta(M3, N3, Delta, prior_sigma_squared, seed)


start_time = time.time()
theta_alg_timeline3 = run_alg(y3, phi_matrix3, K, T, delta, Delta, nu, alpha, prior_sigma_squared)
end_time = time.time()
alg_mse3 = alg_mse(theta3, theta_alg_timeline3)
print("execution time: {:.2f}seconds".format(end_time - start_time))

np.save("./data_saved/normal_theta_origin_Delta0_01_seed212_N1728", theta3)
np.save("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N1728", theta_alg_timeline3)

In [ ]:
theta1_load = np.load("./data_saved/normal_theta_origin_Delta0_01_seed212_N192.npy")
theta2_load = np.load("./data_saved/normal_theta_origin_Delta0_01_seed212_N768.npy")
theta3_load = np.load("./data_saved/normal_theta_origin_Delta0_01_seed212_N1728.npy")

theta_alg_timeline1_load = np.load("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N192.npy")
theta_alg_timeline2_load = np.load("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N768.npy")
theta_alg_timeline3_load = np.load("./data_saved/normal_theta_alg_alpha2_K50_T200_delta0_1_Delta0_01_priorsigma1_seed212_N1728.npy")

alg_mse1_load = alg_mse(theta1_load, theta_alg_timeline1_load)
alg_mse2_load = alg_mse(theta2_load, theta_alg_timeline2_load)
alg_mse3_load = alg_mse(theta3_load, theta_alg_timeline3_load)

In [ ]:
L1 = int(T/delta)
mmse = np.trace(np.linalg.inv(alpha/Delta * np.dot(phi_matrix1.T, phi_matrix1) + np.eye(N1) * 1/prior_sigma_squared))/N1
plot_start = 30
plt.xlabel("T")
plt.ylabel("log MSE")
plt.plot((np.array(range(L1)))[plot_start::10]*delta, np.log(alg_mse1_load[plot_start::10]/2), label="log Algorithm MSE: N=192")
plt.plot((np.array(range(L1)))[plot_start::10]*delta, np.log(alg_mse2_load[plot_start::10]/2), label="log Algorithm MSE: N=768")
plt.plot((np.array(range(L1)))[plot_start::10]*delta, np.log(alg_mse3_load[plot_start::10]/2), label="log Algorithm MSE: N=1728")

plt.plot((np.array(range(L1)))[plot_start::10]*delta, np.log((mmse*(np.ones(L1)))[plot_start::10]), label="log MMSE")
plt.legend()

plt.savefig("./data_saved/normal_amp_mse_Delta0_01.png")